# Exploring new German data

Information from Philipp about the file naming:
    
%proj%=%spec%=%conn%=%tset%=%date%=%ptst%=%test%=%equi%=%tid%

%proj%  Project Title<br>
%spec%  Specimen Name<br>
%conn%  Connection Name<br>
%tset%  Testset Name<br>
%date%  Start Date<br>
%ptst%  Parent Test Name (empty if not existing)<br>
%test%  Test Name<br>
%equi%  Equipment Name<br>
%tid%   Unique Id for Test based on data location
        (available if test was imported to ahjo)<br>


In [1]:
import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import sys
from tqdm import tqdm

In [2]:
# Set up the translation into English for the column headers
# German column headers
german = ["Schritt","Zustand","Zeit","Programmdauer","Schrittdauer","Zyklus",
          "Zyklusebene","Prozedur","Prozedurebene","AhAkku","AhLad","AhEla",
          "AhStep","Energie","WhStep","Spannung","Strom","Temp13"]

# English translations
english = ["step", "state", "time", "programme duration", "step duration",
           "cycle", "cycle level", "procedure", "procedure level", "Qacc",
           "Qcha", "Qdch", "AhStep", "energy", "WhStep", "voltage",
           "current", "temp13"]

# Check list lengths match
assert(len(german) == len(english))

# Create a dictionary and view a test entry
translate = dict(zip(german, english))
print(translate['Zeit'])

time


In [3]:
# Define where the CSV files are stored and get a list of their paths
file_dir = "D:/Dropbox/UoE_Batteries/new_german_data/"
files = glob.glob(file_dir + "**/*.csv", recursive=True)
# Make a list of just the CSV file names (not paths) for easier file locating
csv_names = [file.split("\\")[1] for file in files]


# Specify a converter dictionary for use with pd.read_csv, to specify data types
# of columns contained within the CSV. Use the original German column names.
# Need to find out what to do with the time column. Leave as "object" for now.
dtypes = [int, str, object, float, float, int, int, str, int,
          float, float, float, float, float, float, float, float, float]

converter = dict(zip(german, dtypes))

## Build a function to read the data from a file and handle the translation

In [4]:
def load_from_csv(fpath, converter, fields=None, translation=True): 
    '''
    Load data from CSV files provided by Philipp at Aachen.
    Handle data type conversion, loading a specific set of columns and translating column headers.
    
    Inputs:
        fpath (type: str)
            Path to the CSV file you want to load
        
        converter (type: dict)
            Dict to map data in columns to desired data types.
            Keys: German column names.
            Values: Data type of column
        
        fields (type: list)
            A list of German column names that should be loaded from the CSV file
            
        translation (type: bool, default: True)
            Whether or not to translate the German column headers into English
            (requires a dict called "translate", defined outside the scope of this function)
        
    '''
    
    df = pd.read_csv(fpath, skiprows=[1], header=0, dtype=converter)
    
    # Initialise variable name so there's something to return irrespective of translation bool state
    translation_error_log = None
    
    # Go through translation routine if required. Check for failed translations.
    if translation:
        # Translate German columns where a translation exists in the dictionary, else leave the German.
        translated_cols = [translate[ger_col] if ger_col in translate.keys() else ger_col for ger_col in df.columns]
        
        # Store any column names that haven't been translated
        failed_translations = np.where([col not in translate.values() for col in translated_cols])[0]
        
        # This condition fails if len(failed_translations)==0
        if np.any(failed_translations):
            # Add the file path, as well as all failed translations and their column index
            translation_error_log = [fpath, [(idx, df.columns[idx]) for idx in failed_translations]]
        
        # Replace the column names with the translated names
        df.columns = translated_cols
        
    
    # Get rid of null rows, if present
    df.dropna(inplace=True)
    # Reset the indices in case of null row deletion
    df.reset_index(inplace=True, drop=True)
    
    return df, translation_error_log



In [5]:
# Load an example file using the function
data_from_fn, _ = load_from_csv(files[2], converter, translation=True)

### Test the load_from_csv function and examine translation results
Some files have additional columns. Find all unique column names and see if we need these extra ones. If so, add the German and English to the translate dictionary

In [15]:
# Initialise a list to store filepaths for files that raise an exception inside the function
failed_files = []
# Initialise a set to store unique German column names
unique_columns = set()

for i, f in tqdm(enumerate(files)):
    try:
        # Set translation to False so we get the German column names
        data, trans_error_log = load_from_csv(f, converter, translation=False)
        # Get the German column names and add them to the set
        for col in data.columns:
            # Add every column name from every file to the set
            unique_columns.add(col)
            
    except:
        e = sys.exc_info()[1]
        failed_files.append([f, e])

69it [00:08,  7.91it/s]


In [16]:
# Now let's look at the column names that are not already contained in our "german" list
# Get the column names that are present in both "unique_columns" and "german" variables
intersection = np.intersect1d(list(unique_columns), german)

# Find the column names in "unique_columns" but NOT in "german".
# symmetric_difference is a method of the set class. It returns a set
new_cols = list(unique_columns.symmetric_difference(german))

# We can see that for this first batch of files, at least, these additional columns
# don't seem to be important for us. They are mostly related to temperatures.
# TODO - find out what Agilent is. Translator doesn't work.
print(new_cols)

['Temp23', 'Temp0029', 'ActTemp', 'ClimaEN', 'Temp0028', 'Temp', 'SetTemp', 'Temp0030', 'Agilent', 'ClimaOn', 'Temp0027']


## Look at the data from an example file

In [8]:
data, _ = load_from_csv(files[4], converter, translation=True)
print(f"{len(data)} rows")
print(data.head())

88320 rows
   step state                    time  ...   voltage   current   temp13
0     4   DCH  2013-01-24 16:50:17.26  ...  3.482481 -3.359215  27.4375
1     4   DCH  2013-01-24 16:50:17.34  ...  3.504115 -2.740188  27.4375
2     4   DCH  2013-01-24 16:50:17.34  ...  3.504115 -2.740188  27.4375
3     4   DCH  2013-01-24 16:50:17.39  ...  3.500815 -2.811177  27.4375
4     4   DCH  2013-01-24 16:50:17.66  ...  3.500815 -2.760065  27.4375

[5 rows x 18 columns]


In [7]:
# Find the unique step values
print(data['step'].unique())

# Find the unique state values
print(data['state'].unique())

# Look at the state value for each of these steps
for step_num in data['step'].unique():
    temp_df = data[data['step'] == step_num]
    print(step_num)
    print(temp_df.head(1))
    print()

[   4    5    7 9999]
['DCH' 'CHA' 'STO']
4
   step state                   time  ...  voltage   current   temp13
0     4   DCH  2013-01-16 09:08:22.5  ...  3.52135 -2.433514  24.3125

[1 rows x 18 columns]

5
     step state                    time  ...   voltage   current   temp13
238     5   CHA  2013-01-16 09:38:22.98  ...  3.504076  0.004257  27.0625

[1 rows x 18 columns]

7
       step state                    time  ...   voltage  current   temp13
88787     7   STO  2013-01-23 01:11:23.88  ...  3.898142      0.0  27.6875

[1 rows x 18 columns]

9999
       step state                    time  ...   voltage  current   temp13
88788  9999   STO  2013-01-23 01:11:23.88  ...  3.898142      0.0  27.6875

[1 rows x 18 columns]



In [34]:
# Plot some data
# Get a DataFrame for a CHA step
# cha_df = data[data['state']=='CHA']
# V = cha_df['voltage'].to_numpy()
# I = cha_df['current'].to_numpy()

V = data['voltage'].to_numpy()
I = data['current'].to_numpy()
temp_df = data[data['step'] != 9999]
steps = temp_df['step']



fig, ax1 = plt.subplots()

color = 'tab:red'
#ax1.set_xlabel('time (s)')
#ax1.set_ylabel('Current and Voltage', color=color)
ax1.plot(V, label='Voltage')
ax1.plot(I, label='Current')
ax1.set_ylim([-10, 10])
ax1.legend()
ax1.grid(alpha=0.4)
#ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Step')  # we already handled the x-label with ax1
ax2.set_ylim([0, 200])
ax2.plot(steps, color='black', label='Step', alpha=0.7)
#ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()





# fig, ax = plt.subplots()
# ax.plot(V, label='Voltage')
# ax.plot(I, label='Current')
# ax.plot(steps, label='Step')
# ax.legend()
# ax.grid(alpha=0.5)

# plt.show()

### Get the number of cycles present in the files that contain repeated discharge-charge processes

In [13]:
# Based on the saved plots, we see that the following files are dch-cha repeated
cycle_file_indices = [2, 4, 6, 8, 10, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31]
num_cycles = [None for i in range(len(cycle_file_indices))]
cycles_per_file = dict(zip(cycle_file_indices, num_cycles))

# Load each of the files
for idx in cycle_file_indices:
    data, _ = load_from_csv(files[idx], converter, translation=True)
    cycles_per_file[idx] = np.max(data['cycle'])


In [32]:
# Plot some voltage and current data from one of the files that has repeated cycles
data, _ = load_from_csv(files[2], converter, translation=True)
#subset = data[(data.cycle >= 1) & (data.cycle <= 6)]
subset = data[data['cycle'] == 1]

fig, ax = plt.subplots()
ax.plot(subset['voltage'], color='blue', label='Voltage')
ax.set_ylim([3, 4])
ax.legend(loc=3)

ax2 = ax.twinx() 
ax2.plot(subset['current'], color='orange', label='Current')
ax2.set_ylim([-5, 5])
ax2.legend(loc=4)
ax2.grid(alpha=0.4)

plt.show()


### Let's look at how the current and voltage profiles change over the course of N cycles.
### Each file contains 160 cycles, so we could look at the first cycle from each file and plot them to see if there's a shift

In [134]:
# Get the files for cell 18
cell_18_files = [f for f in files if "018=" in f]
# Make a dictionary to store the current and voltage data
cycle_data = {k: {'V': None, 'I': None} for k in cycle_file_indices}

for idx in list(cycle_data.keys()):
    data, _ = load_from_csv(cell_18_files[idx], converter, translation=True)
    # Take a subset of the DataFrame
    data = data[data['cycle'] == 1]
    cycle_data[idx]['V'] = data['voltage'].to_numpy().astype(float)
    cycle_data[idx]['I'] = data['current'].to_numpy().astype(float)

In [136]:
# Plot the currents
fig, ax = plt.subplots()
for k in list(cycle_data.keys()):
    ax.plot(cycle_data[k]['I'], label=k)

ax.grid(alpha=0.4)    
ax.legend()
plt.show()

In [45]:
# Plot the voltages
fig, ax = plt.subplots()
for k in list(cycle_data.keys()):
    ax.plot(cycle_data[k]['V'], label=k)

ax.grid(alpha=0.4)    
ax.legend()
plt.show()

In [123]:
# Look at deaing with the time information. Interpolate so it's on a regularly spaced time reference
import datetime

data, _ = load_from_csv(cell_18_files[2], converter, translation=True)

In [125]:
# dt1 = datetime.datetime.strptime(data.loc[0]['time'], "%Y-%m-%d %H:%M:%S.%f")
# dt2 = datetime.datetime.strptime(data.loc[1]['time'], "%Y-%m-%d %H:%M:%S.%f")
# elapsed_time = dt2 - dt1
# print(elapsed_time.total_seconds())

# Make a new DataFrame that just contains the time, for experimentation and investigate vectorising the
# function to compute the number of seconds at each measurement
t_data = data['time']
time_df = pd.DataFrame(t_data)
# Create an empty column for the time in seconds, starting at zero
#time_df['t'] = np.nan

# Note - avoid chained indexing e.g. time.loc[0]['t']
#time_df.at[0, 't'] = 0.0

# Get pandas datetime objects
datetimes = pd.to_datetime(time_df['time'])


0.06


In [96]:
def grimace_function(df):
    
    # Get a datetime object for the first entry in the time column
    row1 = df.iloc[0]
    dt1 = datetime.datetime.strptime(row1['time'], "%Y-%m-%d %H:%M:%S.%f")
    
    # Shouldn't use a loop, but do it for experimentation
    for i in df.index:
        if i == 0:
            continue
        dt_val = datetime.datetime.strptime(df.at[i, 'time'], "%Y-%m-%d %H:%M:%S.%f")
        elapsed_time = dt_val - dt1
        elapsed_time = elapsed_time.total_seconds()
        df.at[i, 't'] = elapsed_time
        
    return df
    
    
grimace_function(time_df)

dt1 = datetime.datetime.strptime(time_df['time'].iloc[0], "%Y-%m-%d %H:%M:%S.%f")
dt2 = datetime.datetime.strptime(time_df['time'].iloc[-1], "%Y-%m-%d %H:%M:%S.%f")
elapsed_time = dt2 - dt1
print(elapsed_time.total_seconds())

2013-01-16 09:08:22.500000
576181.38


In [127]:
def datestr_to_datetime(x):
    return datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f")


def timedelta_to_float(x):
    return x.total_seconds()


# Vectorise the functions
convert_time = np.vectorize(datestr_to_datetime)
get_seconds = np.vectorize(timedelta_to_float)


time_df = pd.DataFrame(data['time'])
datetimes = pd.to_datetime(time_df['time'])

result = convert_time(datetimes.astype(str))

# Get the array of times elapsed since the first row's time stamp
elapsed = result - result[0]   # Type is timedelta

time_df['elapsed'] = get_seconds(elapsed)

In [174]:
import time

def datestr_to_datetime(x):
    return datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f")


def timedelta_to_float(x):
    return x.total_seconds()


# Vectorise the functions
convert_time = np.vectorize(datestr_to_datetime)
get_seconds = np.vectorize(timedelta_to_float)


def get_elapsed_time_seconds(df):
    #print("in get_elapsed_time_seconds")
    # Extract the time column and store as a DataFrame
    time_df = pd.DataFrame(df['time'])
    # Convert the entries to pandas Timestamp object
    #print("Converting to pandas Timestamp objects")
    datetimes = pd.to_datetime(time_df['time'])
    # Convert to Python datetime object to use the total_seconds() method
    #print("Converting to python datetime objects")
    py_datetimes = convert_time(datetimes.astype(str))
    # Get the array of times elapsed since the first row's time stamp
    #print("Getting elapsed time")
    elapsed = py_datetimes - py_datetimes[0]   # Type is timedelta
    # Use the vectorised function to convert all the timedelta objects to floats
    seconds_array = get_seconds(elapsed)
    
    return seconds_array


start = time.time()
for idx in cycle_file_indices:
    print(idx)
    # Load the data
    data, _ = load_from_csv(cell_18_files[idx], converter, translation=True)
    # Add a column for elapsed time in seconds
    data['elapsed'] = None

    # Remove instances of cycle number zero - these occur at the end of each file
    data = data[data['cycle'] != 0]
    data.reset_index(inplace=True, drop=True)

    # Get the cycle numbers present
    cycles = data['cycle'].unique()

    for i, cycle in enumerate(cycles):

        select_indices = list(np.where(data["cycle"] == cycle)[0])
        elapsed_array = get_elapsed_time_seconds(data.iloc[select_indices])
        #elapsed_array = get_elapsed_time_seconds(subset)
        data.loc[data.cycle==cycle, 'elapsed'] = elapsed_array
        #data.iloc[select_indices, 'elapsed'] = elapsed_array
    
    
print(time.time() - start)

2
4
6
8
10
13
15
17
19
21
23
25
27
29
31
27.159066200256348


In [180]:
# Make a copy of the DataFrame for just one cycle and experiment with the interpolation
from scipy.interpolate import interp1d

cycle_one = pd.DataFrame(df[df.cycle == 1])

# Get numpy arrays for elapsed time, voltage and current
elapsed_arr = cycle_one['elapsed'].to_numpy()
voltage_arr = cycle_one['voltage'].to_numpy()
current_arr = cycle_one['current'].to_numpy()

# Instantiate a regularly spaced time array
regular_time = np.arange(0, np.max(elapsed_arr), 1)
f_i = interp1d(elapsed_arr, current_arr)
f_v = interp1d(elapsed_arr, voltage_arr)

interp_current = f_i(regular_time)
interp_voltage = f_v(regular_time)

In [186]:
fig, ax = plt.subplots()
ax.plot(regular_time, interp_current, color='blue', label='Interp. current')
ax.plot(elapsed_arr, current_arr, 'o', color='purple', label='Experimental current')
ax.set_ylim([-5, 5])
ax.legend(loc=3)

ax2 = ax.twinx() 
ax2.plot(regular_time, interp_voltage, color='orange', label='Interp. voltage')
ax2.plot(elapsed_arr, voltage_arr, 'o', color='red', label='Experimental voltage')
ax2.set_ylim([3, 4])
ax2.legend(loc=4)
ax2.grid(alpha=0.4)

plt.show()

In [100]:



def grimace_vectorised(df):
    '''
    Write the steps:
    * Create a new column containing datetime objects from the 'time' column
    
    
    
    '''
    
    
    
    
    
    # Get a datetime object for the first entry in the time column
    row1 = df.iloc[0]
    dt1 = datetime.datetime.strptime(row1['time'], "%Y-%m-%d %H:%M:%S.%f")
    
    # Get an array of strings
    string_array = df['time'].to_numpy()
    
    # Try applying datetime to it
    
    print(type(dt1))
    print(type(row1['time']))
    #df = df.assign(date_times = lambda x: )
    #df = df.assign(t_vectorised = lambda x: (datetime.datetime.strptime(x['time'], "%Y-%m-%d %H:%M:%S.%f") - dt1))
    
bilge_df = grimace_vectorised(time_df)

<class 'datetime.datetime'>
<class 'str'>


In [64]:
subset = data[data['cycle'] == 1]
plt.plot(subset['step duration'])

In [38]:
# Find out how many cycles there are
print("Number of cycles: ", len(cha_df['cycle'].unique()))
print("Min. cycle number: ", np.min(cha_df['cycle']))
print("Max cycle number: ", np.max(cha_df['cycle']))

Number of cycles:  1
Min. cycle number:  0
Max cycle number:  0


In [8]:
# Get the data from a particular step
step_44_data = data[data['step']==44]
V = step_44_data['voltage'].to_numpy()
I = step_44_data['current'].to_numpy()
Q = step_44_data['Qcha'].to_numpy()

fig, ax = plt.subplots(1,2)
ax[0].plot(V)
ax[0].plot(I)
ax[1].plot(Q)
plt.show()